In [13]:
from tensorflow import keras
import pandas as pd
import tweepy
import config
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle
from keras.preprocessing.sequence import pad_sequences

In [15]:
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text


tokenizer= pickle.load(open("tokenizer.pkl", "rb"))
maxlen = 130

In [2]:
movie = input("Enter your movie name: ")
movie_hashtag='#'+movie.replace('/n','').replace(' ','')
print("Looking For Tweets: ", movie_hashtag)

Enter your movie name: The Dark Knight
Looking For Tweets:  #TheDarkKnight


In [3]:
client=tweepy.Client(bearer_token=config.BEARER_TOKEN)
query='review '+movie_hashtag+' -is:retweet lang:en -has:media'
response=client.search_recent_tweets(query=query, max_results=10)
print(response)

Response(data=[<Tweet id=1636289877095464961 text='Batman: Everybody Lies review — bat signal missing in action... - Gaming Trend https://t.co/QY8fssTNt8 #batman #warnerbros #dccomics #fumetti #comics #catwoman #robin #damianwayne #batmobile #paninicomics #joker #harleyquinn #brucewayne #thedarkknight #DCFanDome #BatmanDay #Sup…'>, <Tweet id=1636289875304390656 text='Batman: The Adventures Continue Season 3 #3 review - Batman-News https://t.co/PtBnexLy7t #batman #warnerbros #dccomics #fumetti #comics #catwoman #robin #damianwayne #batmobile #paninicomics #joker #harleyquinn #brucewayne #thedarkknight #DCFanDome #BatmanDay #Superman #wonderw…'>, <Tweet id=1635972784613359616 text='Batman Incorporated #6 review - Batman-News https://t.co/NCxATGJ8mG #batman #warnerbros #dccomics #fumetti #comics #catwoman #robin #damianwayne #batmobile #paninicomics #joker #harleyquinn #brucewayne #thedarkknight #DCFanDome #BatmanDay #Superman #wonderwoman #Aquaman #Flash …'>, <Tweet id=163595763139834265

In [4]:
lst_movie_reviews_from_twtr=[]
for tweet in response.data:
    lst_movie_reviews_from_twtr.append(tweet.text)

In [16]:

reconstructed_model = keras.models.load_model("sentiment-score-model")


data_test = {'review':lst_movie_reviews_from_twtr}
df_test = pd.DataFrame(data=data_test)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x: clean_text(x))

list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = reconstructed_model.predict(X_te)
print("prediction:",prediction)

prediction: [[0.0503996 ]
 [0.17345217]
 [0.2706344 ]
 [0.17345217]
 [0.7090525 ]
 [0.524767  ]
 [0.7090525 ]
 [0.61247104]
 [0.61247104]]


In [17]:
df_test['Sentiment Score']=prediction
df_test

,review,Sentiment Score
0,Batman: Everybody Lies review — bat signal mis...,0.050400
1,Batman: The Adventures Continue Season 3 #3 re...,0.173452
2,Batman Incorporated #6 review - Batman-News ht...,0.270634
3,Batman: The Adventures Continue Season 3 #3 re...,0.173452
4,Review: Batman: The Doom That Came Gotham (202...,0.709053
5,'Gotham Knights' 'Superman &amp; Lois' give CW...,0.524767
6,Review: Batman: The Doom That Came Gotham (202...,0.709053
7,Batman: The Doom That Came Gotham (2023) Revie...,0.612471
8,Batman: The Doom That Came Gotham (2023) Revie...,0.612471


In [18]:
p_mean=prediction.mean()
if p_mean>=0.6:
    print('Must Watch','Average Review Score:',p_mean*100,'%')
if p_mean>0.3 and p_mean<0.6:
    print('Maybe Watch','Average Review Score:',p_mean*100,'%')
if p_mean<=0.3:
    print('Not Worth It','Average Review Score:',p_mean*100,'%')

Maybe Watch Average Review Score: 42.61947274208069 %


In [26]:
print('pandas==',pd.__version__)
print('tweepy==',tweepy.__version__)
print('keras==',keras.__version__)
print('nltk==',nltk.__version__)
##print('pickle==',pickle.__version__)
print('tensorflow==',nltk.__version__)
print('re==',re.__version__)

pandas== 1.3.4
tweepy== 4.8.0
keras== 2.8.0
nltk== 3.6.5
tensorflow== 3.6.5
re== 2.2.1


In [28]:
!python -m pip show pickle

4.8.0


In [23]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle
from keras.preprocessing.sequence import pad_sequences
import nltk
import tensorflow